In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import matplotlib.ticker as mticker
import time
import csv
import dateutil.parser, dateutil.tz as tz
import zoneinfo

In [ ]:
categories = ["All_Beauty", "Amazon_Fashion", "Appliances", "Arts_Crafts_and_Sewing", "Automotive", "Baby_Products", "Beauty_and_Personal_Care", "Books", "CDs_and_Vinyl", "Cell_Phones_and_Accessories", "Clothing_Shoes_and_Jewelry", "Digital_Music", "Electronics", "Gift_Cards", "Grocery_and_Gourmet_Food", "Handmade_Products", "Health_and_Household", "Health_and_Personal_Care", "Home_and_Kitchen", "Industrial_and_Scientific", "Kindle_Store", "Magazine_Subscriptions", "Movies_and_TV", "Musical_Instruments", "Office_Products", "Patio_Lawn_and_Garden", "Pet_Supplies", "Software", "Sports_and_Outdoors", "Subscription_Boxes", "Tools_and_Home_Improvement", "Toys_and_Games", "Video_Games", "Unknown"]
data_fn = '../data/amazon_reviews/All_Beauty.csv'
headers = ["user_id","parent_asin","rating","timestamp"]
# df = pd.read_csv(data_fn, header=None, names=headers)
# display(df)

In [3]:
def timestamp_to_timeseries(timestamps, start_time, end_time, bin_size):
    hist, _ = np.histogram(timestamps, bins=np.arange(start_time, end_time+bin_size, bin_size))
    return hist

def groupby(df, group_header='category_id', timestamp_header='timestamp'):
    np_df = df[[group_header, timestamp_header]].to_numpy()
    groups, groups_index = np.unique(np_df[:, 0], return_index=True)
    return np.split(np_df[:,1], groups_index[1:]), groups

def timeseries_to_csv(timeseries, group, csv_fn):
    timeseries_group = [group]
    timeseries_group.extend(list(timeseries))
    with open(csv_fn, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(timeseries_group)

In [7]:
# df['time']=pd.to_datetime(df['timestamp'],unit='s')+datetime.timedelta(hours=8)
start_time = datetime.datetime(1996, 5, 1, 00, 00).timestamp()
end_time = datetime.datetime(2023, 9, 30, 00, 00).timestamp()
bin_size = 60*60 # in seconds
csv_fn = '../data/amazon_reviews_hourly.csv'
timestamp_header = 'timestamp'
group_header = 'category'
print(start_time, end_time)

# timeseries_to_csv(np.arange(start_time, end_time+bin_size, bin_size), group_header, csv_fn)
# timestamps_grouped, groups= groupby(df, group_header, timestamp_header)
# for timestamps, group in zip(timestamps_grouped, groups):
#     timeseries = timestamp_to_timeseries(timestamps, start_time, end_time, bin_size)
#     timeseries_to_csv(timeseries, group, csv_fn)

1602133857705

830934000.0 1696057200.0


1602133857705

In [8]:
timeseries_to_csv(np.arange(start_time, end_time+bin_size, bin_size), group_header, csv_fn)
for category in categories:
    data_fn = f'../data/amazon_reviews/{category}.csv'
    df = pd.read_csv(data_fn, header=0) 
    np_df = df[timestamp_header].to_numpy() // 1000
    hist = timestamp_to_timeseries(np_df, start_time, end_time, bin_size)
    timeseries_to_csv(hist, category, csv_fn)